In [1]:
import geojson
import json
import requests
import os
from arcgis.features import FeatureLayerCollection
from arcgis.gis import GIS
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

In [2]:
#setup JSONBin.io 
JSONBIN_ACCESS_KEY = '$2a$10$J23yWei2a5JsxL9JkiWOXuZsgZ.qR/GRj74Jre.4i/Te3XjaT0A2y'
JSONBIN_API_URL = f'https://api.jsonbin.io/v3/b/66db5907acd3cb34a87f7d42'

# Fetch data from JSONBin
headers = {'X-Master-Key': JSONBIN_ACCESS_KEY}
response = requests.get(JSONBIN_API_URL, headers=headers)

if response.status_code == 200:
    json_data = response.json()


     # Extract 'features'
    feature_collection = json_data.get("record", {}).get("features", [])
    # Filter out features where "text" field contains "test"
    filtered_features = [feature for feature in feature_collection if feature.get("datetime")]
    
    # Update the JSON structure
    json_data["record"]["features"] = filtered_features
    
    # Update JSONBin
    update_response = requests.put(JSONBIN_API_URL, headers=headers, json=json_data)
    
    if update_response.status_code == 200:
        print("Successfully updated JSONBin.")
    else:
        print(f"Failed to update JSONBin: {update_response.status_code}, {update_response.text}")
else:
    raise Exception(f"Failed to fetch data: {response.status_code}, {response.text}")

Successfully updated JSONBin.


In [9]:
# JSONBin details
API_KEY = '$2a$10$J23yWei2a5JsxL9JkiWOXuZsgZ.qR/GRj74Jre.4i/Te3XjaT0A2y' 
JSONBIN_URL = f'https://api.jsonbin.io/v3/b/66db5907acd3cb34a87f7d42'

# Read the Excel file
excel_file_path = r"C:\GreenGIS\NearMissDash\scripts\events2_26_25.xlsx"
df = pd.read_excel(excel_file_path)

# Ensure the required columns exist in the DataFrame
required_columns = [
    "Longitude", "Latitude", "report_type", "event_type", "travel_type", "injury_type",
    "bhrs1", "bhrs2", "bhrs3", "bhrs4", "bhrs5", "bhrs6", "bhrs7", "bhrs8",
    "loc1", "loc2", "loc3", "loc4", "loc5", "loc6", "loc7", "loc8", "loc9",
    "loc10", "loc11", "loc12", "loc13", "text", "datetime"
]

# Validate column presence
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")

# Replace NaN, None, inf, -inf with valid values
df = df.replace([np.nan, None, np.inf, -np.inf], 0)

# Convert DataFrame to GeoJSON FeatureCollection
features = []
for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [float(row["Longitude"]), float(row["Latitude"])]
        },
        "properties": {
            "report_type": str(row["report_type"]),
            "event_type": str(row["event_type"]),
            "travel_type": str(row["travel_type"]),
            "injury_type": str(row["injury_type"]),
            "bhrs1": int(row["bhrs1"]), "bhrs2": int(row["bhrs2"]), "bhrs3": int(row["bhrs3"]), "bhrs4": int(row["bhrs4"]),
            "bhrs5": int(row["bhrs5"]), "bhrs6": int(row["bhrs6"]), "bhrs7": int(row["bhrs7"]), "bhrs8": int(row["bhrs8"]),
            "loc1": int(row["loc1"]), "loc2": int(row["loc2"]), "loc3": int(row["loc3"]), "loc4": int(row["loc4"]),
            "loc5": int(row["loc5"]), "loc6": int(row["loc6"]), "loc7": int(row["loc7"]), "loc8": int(row["loc8"]),
            "loc9": int(row["loc9"]), "loc10": int(row["loc10"]), "loc11": int(row["loc11"]), "loc12": int(row["loc12"]),
            "loc13": int(row["loc13"]),
            "text": str(row["text"]),
            "datetime": str(row["datetime"])
        }
    }
    features.append(feature)

# Construct GeoJSON FeatureCollection
feature_collection = {
    "record": {
        "type": "FeatureCollection",
        "features": features
    }
}

# Print preview of the JSON structure
print("\nFeatureCollection Preview:")
print(json.dumps(feature_collection, indent=4))

# Fetch the existing JSON data from JSONBin
headers = {
    "X-Master-Key": API_KEY,
    "Content-Type": "application/json"
}

response = requests.get(JSONBIN_URL, headers=headers)

if response.status_code == 200:
    json_data = response.json()

    # Update JSON with the new FeatureCollection
    json_data["record"] = feature_collection["record"]

    # Update JSONBin
    update_response = requests.put(JSONBIN_URL, headers=headers, json=json_data)

    if update_response.status_code == 200:
        print("\nSuccessfully updated JSONBin with FeatureCollection.")
    else:
        print(f"\nFailed to update JSONBin: {update_response.status_code}, {update_response.text}")
else:
    raise Exception(f"\nFailed to fetch data: {response.status_code}, {response.text}")


FeatureCollection Preview:
{
    "record": {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [
                        -77.56475249803376,
                        37.6374734793625
                    ]
                },
                "properties": {
                    "report_type": "Near Miss",
                    "event_type": "Collision2",
                    "travel_type": "Bicycle",
                    "injury_type": "No Inury",
                    "bhrs1": 1,
                    "bhrs2": 0,
                    "bhrs3": 0,
                    "bhrs4": 1,
                    "bhrs5": 0,
                    "bhrs6": 0,
                    "bhrs7": 0,
                    "bhrs8": 0,
                    "loc1": 0,
                    "loc2": 0,
                    "loc3": 0,
                    "loc4": 0,
               

In [17]:
excel_file_path = r"C:\GreenGIS\NearMissDash\scripts\events2_26_25.xlsx"
# Convert DataFrame to GeoJSON FeatureCollection format
features = []
for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["Longitude"], row["Latitude"]]  # Adjust column names
        },
        "properties": {
            "report_type": row["report_type"],
            "event_type": row["event_type"],
            "travel_type": row["travel_type"],
            "injury_type": row["injury_type"],
            "bhrs1": row["bhrs1"],
            "bhrs2": row["bhrs2"],
            "bhrs3": row["bhrs3"],
            "bhrs4": row["bhrs4"],
            "bhrs5": row["bhrs5"],
            "bhrs6": row["bhrs6"],
            "bhrs7": row["bhrs7"],
            "bhrs8": row["bhrs8"],
            "loc1":  row["loc1"],
            "loc2": row["loc2"],
            "loc3": row["loc3"],
            "loc4": row["loc4"],
            "loc5": row["loc5"],
            "loc6": row["loc6"],
            "loc7": row["loc7"],
            "loc8": row["loc8"],
            "loc9": row["loc9"],
            "loc10": row["loc10"],
            "loc11": row["loc11"],
            "loc12": row["loc12"],
            "loc13": row["loc13"],
            "text": row["text"],
            "datetime": row["datetime"]
        }
    }
    features.append(feature)

# Construct GeoJSON structure
feature_collection = {
    "type": "FeatureCollection",
    "features": features
}

# Print preview of the JSON structure
print("\nFeatureCollection Preview:")
print(json.dumps(feature_collection, indent=4))

# Fetch the existing JSON data
headers = {"X-Master-Key": JSONBIN_ACCESS_KEY}
response = requests.get(JSONBIN_API_URL, headers=headers)

if response.status_code == 200:
    json_data = response.json()

    # Update JSON with the new FeatureCollection
    json_data["record"]["feature_collection"] = feature_collection

    # Update JSONBin
    update_response = requests.put(JSONBIN_API_URL, headers=headers, json=json_data)

    if update_response.status_code == 200:
        print("\nSuccessfully updated JSONBin with FeatureCollection.")
    else:
        print(f"\nFailed to update JSONBin: {update_response.status_code}, {update_response.text}")
else:
    raise Exception(f"\nFailed to fetch data: {response.status_code}, {response.text}")


FeatureCollection Preview:
{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -77.56475249803376,
                    37.6374734793625
                ]
            },
            "properties": {
                "report_type": "Near Miss",
                "event_type": "Collision2",
                "travel_type": "Bicycle",
                "injury_type": "No Inury",
                "bhrs1": 1,
                "bhrs2": 0,
                "bhrs3": 0,
                "bhrs4": 1,
                "bhrs5": 0,
                "bhrs6": 0,
                "bhrs7": 0,
                "bhrs8": 0,
                "loc1": 0,
                "loc2": 0,
                "loc3": 0,
                "loc4": 0,
                "loc5": 0,
                "loc6": 0,
                "loc7": 0,
                "loc8": 0,
                "loc9": 0,
        

InvalidJSONError: Out of range float values are not JSON compliant

In [6]:
# Convert JSON to GeoDataFrame
features = json_data["record"]["features"]
geometry = [Point(feature["geometry"]["coordinates"]) for feature in features]

# Create GeoDataFrame with a specified CRS (WGS 84)
gdf = gpd.GeoDataFrame(features, geometry=geometry, crs="EPSG:4326")

# Save as a Shapefile
gdf.to_file("output.shp")

print("Shapefile created successfully with CRS EPSG:4326!")

Shapefile created successfully with CRS EPSG:4326!


c:\Users\egreenwell\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value '{'report_type': 'Near Miss', 'event_type': 'Collision2', 'travel_type': 'Bicycle', 'injury_type': 'No Inury', 'bhrs1': 1, 'bhrs2': 0, 'bhrs3': 0, 'bhrs4': 1, 'bhrs5': 0, 'bhrs6': 0, 'bhrs7': 0, 'bhrs8': 0, 'loc1': 0, 'loc2': 0, 'loc3': 0, 'loc4': 0, 'loc5': 0, 'loc6': 0, 'loc7': 0, 'loc8': 0, 'loc9': 0, 'loc10': 0, 'loc11': 0, 'loc12': 0, 'loc13': 0, 'text': 'Car did not give legal 3-foot ROW', 'datetime': '2024-09-15T13:49'}' of field properties has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(


In [11]:
# Extract features
features = json_data["record"]["features"]

# Transform JSON data into a DataFrame
data = []
for feature in features:
    coordinates = feature["geometry"]["coordinates"]
    properties = feature["properties"]
    row = {
        "Longitude": coordinates[0],
        "Latitude": coordinates[1]
    }
    row.update(properties)
    data.append(row)

df = pd.DataFrame(data)

# Save to Excel
df.to_excel("output.xlsx", index=False)

print("Excel file 'output.xlsx' created successfully!")



Excel file 'output.xlsx' created successfully!


In [ ]:
# ArcGIS Enterprise credentials and URL
portal_url = "https://planrva.maps.arcgis.com/home/index.html"  # Replace with your ArcGIS Enterprise URL
username = "egreenwell_PlanRVA"
password = "HOPPER!02isgreat"

# Authenticate with ArcGIS Enterprise
gis = GIS(portal_url, username, password)

# Step 1: Create an empty feature service
service_properties = {
    "title": "Near Miss Events",
    "serviceDescription": "Feature Service for near-miss events created from JSONBin data",
    "capabilities": "Create,Query,Update,Delete,Editing",  # Enable editing if needed
    "spatialReference": {"wkid": 4326},
    "description": "Feature Service created from JSONBin data",
}

# Create the service
feature_service = gis.content.create_service(name="Near Miss Events", service_type="featureService", create_params=service_properties)
print(f"Feature Service Created: {feature_service.title}")

# Step 2: Access the layers in the new service
feature_layer_collection = FeatureLayerCollection.fromitem(feature_service)

# Define layer properties
layer_properties = {
    "name": "Near Miss Events Layer",
    "geometryType": "esriGeometryPoint",
    "fields": [
        {"name": "report_type", "type": "esriFieldTypeString", "alias": "Report Type", "length": 255},
        {"name": "event_type", "type": "esriFieldTypeString", "alias": "Event Type", "length": 255},
        {"name": "travel_type", "type": "esriFieldTypeString", "alias": "Travel Type", "length": 255},
        {"name": "injury_type", "type": "esriFieldTypeString", "alias": "Injury Type", "length": 255},
        {"name": "datetime", "type": "esriFieldTypeDate", "alias": "Datetime"},
        {"name": "text", "type": "esriFieldTypeString", "alias": "Description", "length": 255},
    ],
    "spatialReference": {"wkid": 4326},
}

# Add a layer to the service
feature_layer_collection.manager.add_to_definition({"layers": [layer_properties]})
print(f"Layer added to Feature Service: {feature_layer_collection.layers[0].properties.name}")

# Step 3: Add features to the layer
feature_layer = feature_layer_collection.layers[0]

# Prepare features from the JSON data
features = []
for feature in feature_list:
    geometry = feature.get("geometry")
    attributes = feature.get("properties")
    if geometry and attributes:
        features.append({"geometry": geometry, "attributes": attributes})

# Add features to the layer
feature_layer.edit_features(adds=features)
print("Features successfully added to the layer.")

# Final output: Service URL
print(f"Feature Service URL: {feature_service.url}")